In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.stats import boxcox

from imp import load_source

from sklearn.decomposition import PCA

import aggregate_player_stats as aps

query_strings = load_source('query_strings', '../stats_generation/query_strings.py' )
gps = load_source(  'generate_player_stats', '../stats_generation/generate_player_stats.py' )

In [2]:
%matplotlib inline
plt.style.use('dark_background')
from pylab import rcParams
rcParams['figure.figsize'] = 14, 10

Generate stuff for whole team

In [8]:
reload(aps)
team_stats_df = aps.generate_full_team_aggregate( 2017 )

In [9]:
print team_stats_df.shape
print team_stats_df.columns.values

(4096, 71)
[u'team' 'opp_team' u'week' 'year' 'score' 'opp_score' 'home_flag'
 'away_flag' 'tds' u'rush_att' u'rush_yds' u'pass_att' u'pass_yds'
 u'pass_cmp' u'pass_sack' u'kickoffs' u'punts' u'fg_miss' u'kickret_tds'
 u'punt_ret_tds' u'off_fumb_tot' u'kickoff_in_20' u'kickoff_touchback'
 u'fg_yds' u'fg_made' u'def_int_tds' u'def_frec_tds' u'def_misc_tds'
 u'def_safety' u'def_tkl' u'def_tkl_contributers' u'def_fumb_rec'
 u'def_fumb_forced' u'def_int' u'def_tkl_loss' u'def_sack' u'def_pass_def'
 'score_prev_4' 'opp_score_prev_4' 'home_flag_prev_4' 'away_flag_prev_4'
 'tds_prev_4' 'rush_att_prev_4' 'rush_yds_prev_4' 'pass_att_prev_4'
 'pass_yds_prev_4' 'pass_cmp_prev_4' 'pass_sack_prev_4' 'kickoffs_prev_4'
 'punts_prev_4' 'fg_miss_prev_4' 'kickret_tds_prev_4' 'punt_ret_tds_prev_4'
 'off_fumb_tot_prev_4' 'kickoff_in_20_prev_4' 'kickoff_touchback_prev_4'
 'fg_yds_prev_4' 'fg_made_prev_4' 'def_int_tds_prev_4'
 'def_frec_tds_prev_4' 'def_misc_tds_prev_4' 'def_safety_prev_4'
 'def_tkl_prev_4'

Defense needs to focus on defensive stats, and some small offensive stats from previous opponents

Sacks: 1 point.

Interceptions: 2 points.

Fumbles Recovered: 2 points.

Safeties: 2 points.

Defensive Touchdowns: 6 points.

Kick and Punt Return Touchdowns: 6 points.

2-Point Conversion Returns: 2 points.

Points Allowed (0): 10 points

Points Allowed (1-6): 7 points

Points Allowed (7-13): 4 points

Points Allowed (14-20): 1 points

Points Allowed (21-27): 0 points

Points Allowed (28-34): -1 points

Points Allowed (35+): -4 points

In [21]:
off_stats = [
                'tds',           # May be useful to keep
                u'rush_att', 
                u'rush_yds',     # May be useful
                u'pass_att', 
                u'pass_yds',     # May be useful
                u'pass_cmp', 
                u'pass_sack', 
                u'kickoffs',
                u'punts', 
                u'fg_miss', 
                u'off_fumb_tot',
                u'kickoff_in_20', 
                u'kickoff_touchback', 
                u'fg_yds', 
                u'fg_made' ]     # May be useful

keep_list = ['tds','rush_yds','pass_yds','fg_made']

def_df = team_stats_df[ keep_list ].copy()

In [22]:
# def_frec?
def_stats = [
                u'def_int_tds',          # all_def_tds
                u'def_frec_tds',         # all_def_tds
                u'def_misc_tds' ,        # all_def_tds
                u'def_safety' ,          # def_safety
                u'def_tkl',
                u'def_tkl_contributers', 
                u'def_fumb_forced',
                u'def_fumb_rec',         # all_def_turn
                u'def_int' ,             # all_def_turn
                u'def_tkl_loss',
                u'def_sack' ,            # def_sack
                u'def_pass_def',
                u'kickret_tds',          # all_def_tds
                u'punt_ret_tds'          # all_def_tds
            ]


# All scored the same, hard to predict but can be useful en bulk
def_df['all_def_tds'] = team_stats_df[[
                                        'def_int_tds',
                                        'def_frec_tds',
                                        'def_misc_tds',
                                        'kickret_tds',
                                        'punt_ret_tds'
                                      ]].sum(axis=1)

def_df['all_def_turn'] = team_stats_df[[
                                        'def_fumb_rec',
                                        'def_int'
                                       ]].sum(axis=1)
def_df['def_safety'] = team_stats_df['def_safety']
def_df['def_sack'  ] = team_stats_df['def_sack'  ]

In [33]:
# Aggregate offensive stats are useful, for turning into OPPOSING stats
off_stats2 = [
                'score_prev_4' ,     # opp_
                'opp_score_prev_4' , # def_
                'home_flag_prev_4' , # def_
                'away_flag_prev_4',  # def_
                'tds_prev_4' ,       # opp_
                'rush_att_prev_4' ,
                'rush_yds_prev_4' ,  # opp_
                'pass_att_prev_4',
                'pass_yds_prev_4' ,  # opp_
                'pass_cmp_prev_4' ,
                'pass_sack_prev_4',  # def_
                'kickoffs_prev_4',
                'punts_prev_4' ,
                'fg_miss_prev_4' ,
                'off_fumb_tot_prev_4' ,
                'kickoff_in_20_prev_4' ,
                'kickoff_touchback_prev_4',
                'fg_yds_prev_4' ,
                'fg_made_prev_4'    # opp_
             ]
def_agg = ['opp_score_prev_4','home_flag_prev_4','away_flag_prev_4','pass_sack_prev_4']
opp_agg = ['score_prev_4','tds_prev_4','rush_yds_prev_4','pass_yds_prev_4','fg_made_prev_4']

def_df[def_agg] = team_stats_df[def_agg].copy()
def_df['def_opp_score_prev_4'] = def_df['opp_score_prev_4']
def_df = def_df.drop( 'opp_score_prev_4', axis=1 )

def_df[opp_agg] = team_stats_df[opp_agg].copy()
def_df.rename(columns=dict(zip( opp_agg, ['opp_'+x for x in opp_agg] )), inplace=True)

In [36]:
# Aggregate is good indicator for team
def_stats2 = [
                'kickret_tds_prev_4', 
                'punt_ret_tds_prev_4',
                'def_int_tds_prev_4',
                'def_frec_tds_prev_4', 
                'def_misc_tds_prev_4', 
                'def_safety_prev_4',
                'def_tkl_prev_4', 
                'def_tkl_contributers_prev_4', 
                'def_fumb_rec_prev_4',
                'def_fumb_forced_prev_4', 
                'def_int_prev_4', 
                'def_tkl_loss_prev_4',
                'def_sack_prev_4', 
                'def_pass_def_prev_4', 
                'includes_preseason'    
             ]

# All scored the same, hard to predict but can be useful en bulk
def_df['all_def_tds_prev_4'] = team_stats_df[[
                                        'def_int_tds_prev_4',
                                        'def_frec_tds_prev_4',
                                        'def_misc_tds_prev_4',
                                        'kickret_tds_prev_4',
                                        'punt_ret_tds_prev_4'
                                      ]].sum(axis=1)

def_df['all_def_turn_prev_4'] = team_stats_df[[
                                        'def_fumb_rec_prev_4',
                                        'def_int_prev_4'
                                       ]].sum(axis=1)
def_df['def_safety_prev_4'] = team_stats_df['def_safety_prev_4']
def_df['def_sack_prev_4'  ] = team_stats_df['def_sack_prev_4'  ]

In [37]:
def_df.head()

,tds,rush_yds,pass_yds,fg_made,all_def_tds,all_def_turn,def_safety,def_sack,home_flag_prev_4,away_flag_prev_4,...,def_opp_score_prev_4,opp_score_prev_4,opp_tds_prev_4,opp_rush_yds_prev_4,opp_pass_yds_prev_4,opp_fg_made_prev_4,all_def_tds_prev_4,all_def_turn_prev_4,def_safety_prev_4,def_sack_prev_4
1036,1,40,288,3,0,1,0,3.0,2.0,2.0,...,100.0,53.0,6.0,310.0,1206.0,4.0,0.0,1.0,0.0,9.0
1037,4,118,265,1,1,3,0,4.0,3.0,1.0,...,100.0,59.0,6.0,280.0,1213.0,6.0,0.0,2.0,0.0,12.0
1038,1,24,339,1,0,1,0,0.0,2.0,2.0,...,100.0,84.0,10.0,334.0,1245.0,5.0,1.0,5.0,0.0,11.0
1039,4,44,302,0,1,1,0,0.0,2.0,2.0,...,87.0,57.0,6.0,250.0,1085.0,5.0,1.0,6.0,0.0,9.0
1040,3,62,292,2,0,2,0,5.0,3.0,1.0,...,89.0,85.0,10.0,226.0,1194.0,5.0,2.0,6.0,0.0,7.0
